In [2]:
import pandas as pd
import numpy as np
import json
# import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from datetime import datetime
import re
import sklearn
import seaborn as sns
import collections
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_excel('../Data/OriginalData/TMDB_dataset.xlsx')
df = df.dropna()
df.reset_index(drop=True, inplace=True)

**<span style="font-size:2em;">Popularity Metrics</span>**

**Row-Wise Function to Compute Average Popularity for a Movie Crew/Cast**

In [3]:
def avg_population(row):
    score = 0
    remove_duplicates = []
    if isinstance(row, float):
        return np.nan
    else:
        for person in row.split(','):
            if person not in remove_duplicates:
                remove_duplicates.append(person)
        for m in remove_duplicates:
            if len(m.lstrip().split(':')) > 1:
                score += float(m.lstrip().split(':')[-1])
        return score/len(remove_duplicates)

**Function Call**

In [4]:
df['cast_average'] = df['cast'].apply(lambda c: avg_population(c))
df['crew_average'] = df['crew'].apply(lambda c: avg_population(c))

**<span style="font-size:2em;">Encodings</span>**

**OneHot Encode the Genres**  
Note: Cannot use built in function because we need to change the format

In [5]:
unique_genres = []
for list_of_genres in df['genres'].tolist():
    for genre in list_of_genres.split(','):
        if genre.lstrip() not in unique_genres:
            unique_genres.append(genre.lstrip())

In [6]:
for genre in unique_genres:
    df[genre] = df['genres'].apply(lambda genre_list: 1 if genre in genre_list.split(', ') else 0)
df = df.drop('genres', axis=1)

**OneHot Encode the Production Companies (May not use this attribute because there are 9313 companies)**  
Note: Cannot use built in function because we need to change the format

In [7]:
unique_prod_comp = []
for list_of_comp in df['production_companies'].tolist():
    for comp in list_of_comp.split(','):
        # if comp.lstrip() not in unique_prod_comp:
        unique_prod_comp.append((comp.lstrip()))

In [8]:
common_pc = [c[0] for c in collections.Counter(unique_prod_comp).most_common(25)]
for comp in common_pc:
    df[comp] = df['production_companies'].apply(lambda comp_list: 1 if comp in comp_list.split(', ') else 0)

**OneHot Encode the Original Langauge**  
Note: Also may leave out because there are 50 different languages

In [9]:
df = pd.get_dummies(df, columns=['original_language'])

**Save to New File**

In [10]:
df.to_excel('../Data/ProcessedData/TMDB_processed.xlsx', index=False)